<h1>Dataset Exploration</h1>

In [1]:
from pycocotools.coco import COCO
import matplotlib
import matplotlib.pyplot as plt
import os
import open3d as o3d
import cv2
import numpy as np
from PIL import Image

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
DATADIR = "cocodoom/"

dataSplit, run = "run-full-val", "run2"

annFile = '{}{}.json'.format(DATADIR,dataSplit)

In [3]:
coco = COCO(annFile)

loading annotations into memory...
Done (t=15.63s)
creating index...
index created!


In [4]:
player_positions = {"run1":[], "run2":[], "run3":[]}
motion_vectors = {"run1":[], "run2":[], "run3":[]}
USED_RUNS = ["run2"]
for run in USED_RUNS:
    with open(DATADIR+run+"/log.txt", 'r') as log_file:
        for line in log_file:
            if "player" in line:
                line = line.strip()
                tic, stats = line.split("player:")
                x, y, z, angle = stats.split(",")
    
                # Store position in the dictionary
                player_positions[run].append((float(x), float(y), float(z), float(angle)))
                if len(player_positions[run]) >= 2:
                    player_position = player_positions[run][-1]
                    prev_player_position = player_positions[run][-2]
                    
                    dx = player_position[0] - prev_player_position[0]
                    dy = player_position[1] - prev_player_position[1]
                    dz = player_position[2] - prev_player_position[2]
                    dangle = np.pi - abs(abs(player_position[3] - prev_player_position[3]) - np.pi)
                    
                    dx_relative = dx * np.cos(2 * np.pi - prev_player_position[3]) + dy * np.cos(prev_player_position[3] - 1/2 * np.pi)
                    dy_relative = dx * np.sin(2 * np.pi - prev_player_position[3]) + dy * np.sin(prev_player_position[3] - 1/2 * np.pi)
                    motion_vector = (dx_relative, dy_relative, dz, dangle)
                    motion_vectors[run].append(motion_vector)

In [5]:
def getSegmentationMask(rgb_filename):
    return rgb_filename.replace("rgb", "objects")

def getDepthMask(rgb_filename):
    return rgb_filename.replace("rgb", "depth")

In [20]:
def toPointCloud(depth):
    # https://github.com/mmatl/pyrender/issues/14#issuecomment-485881479 was used as reference
    height, width = depth.shape

    fy = 200 / np.tan(1.5708 * 0.5)
    fx = 320 / np.tan(1.5708 * 0.5)

    mask = np.where(depth > 0)

    x = mask[1]
    y = mask[0]

    normalized_x = (x.astype(np.float32) - width * 0.5) #/ width
    normalized_y = (y.astype(np.float32) - height * 0.5) #/ height
    
    world_x = normalized_x * depth[y, x] / fx #* 1000
    world_y = normalized_y * depth[y, x] / fy #* 1000
    world_z = depth[y, x]
    ones = np.ones(world_z.shape[0], dtype=np.float32)

    return np.vstack((world_x, world_y, world_z)).T

In [21]:
def visiblePoints(point_cloud, motion_vector):
    angle = motion_vector[3]
    rotation_matrix = np.array([[np.cos(angle), -1 * np.sin(angle), 0],[np.sin(angle), np.cos(angle), 0],[0, 0, 1]])
    print(translated_points.shape)
    translated_points = point_cloud - np.array([motion_vector[0], motion_vector[1], motion_vector[2]])
    print(translated_points.shape)
    print(rotation_matrix.shape)
    transformed_points = translated_points @ rotation_matrix

    fy = 200 / np.tan(1.5708 * 0.5)
    fx = 320 / np.tan(1.5708 * 0.5)
    x_proj = transformed_points[:, 0] * fx / transformed_points[:, 2]
    y_proj = transformed_points[:, 1] * fy / transformed_points[:, 2]

    print("Made it here")

    output = np.zeros((200, 320)) # These might need to be switched
    visible_point = []
    for i, z in enumerate(transformed_points[:, 2]):
        if z > 0 and 0 <= x_proj[i] < resolution[0] and 0 <= y_proj[i] < resolution[1]:
            output[y_proj[i], x_proj[i]] = 1
            visible_point.append([x_proj[i], y_proj[i], z])

    return output, transformed_points, visible_point

In [23]:
imgIds = coco.getImgIds();
img = coco.loadImgs(imgIds[np.random.randint(0,len(imgIds))])[0]
d = plt.imread(DATADIR + getDepthMask(img["file_name"]))

print(p:=toPointCloud(d))
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(p)
o3d.io.write_point_cloud("./data.xyz", pcd)

[[-0.04930208 -0.04930208  0.0986038 ]
 [-0.04899395 -0.04930208  0.0986038 ]
 [-0.04868581 -0.04930208  0.0986038 ]
 ...
 [ 0.0317427   0.03202575  0.06469826]
 [ 0.03194488  0.03202575  0.06469826]
 [ 0.03214706  0.03202575  0.06469826]]


True

In [14]:
print(200 / np.tan(1.5708 * 0.5))
print(320 / np.tan(1.5708 * 0.5))

199.99926536032856
319.9988245765257


In [10]:
visibility_matrix, transformed_points, visible_point = visiblePoints(p, [-97.2494, 782.437, 97.1546, 1.5708])

UnboundLocalError: local variable 'translated_points' referenced before assignment